# Notebook 2 for Capstone assignment

*This notebook will be used for the Coursera Capstone assignment*

## Exploring Toronto
### Segmenting and Clustering Neighborhoods in Toronto assignment

Install Beautiful Soup 4 to scrape wiki page and link to web page

In [1]:
# import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib import request

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

content = urlopen(url).read()
soup = BeautifulSoup(content)

print ("Document loaded into Beautiful Soup: ", soup.title.string)


Document loaded into Beautiful Soup:  List of postal codes of Canada: M - Wikipedia


Upload wiki table into a csv file, dressing 'Not assigned' data

In [2]:
req = urlopen(url)
article = req.read().decode()

with open('ISO_3166-1_alpha-2.html', 'w') as fo:
    fo.write(article)

# Load article, turn into soup and get the table.
article = open('ISO_3166-1_alpha-2.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:5] == ['Postcode', 'Borough', 'Neighbourhood', 'ISO 3166-2']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('iso_3166-1_alpha-2_codes.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode, Borough, Neighbourhood = [td.text.strip() for td in tds[:3]]

        # extract rows where borough unassigned and use borogh name where Neighbourhood unassigned
        if 'Not assigned' in Borough:
            continue 
        if 'Not assigned' in Neighbourhood:
            Neighbourhood = Borough
        print('; '.join([Postcode, Borough, Neighbourhood]), file=fo)

Read csv file into dataframe

In [3]:
df = pd.read_csv('iso_3166-1_alpha-2_codes.txt', sep=';', header=None, names=['Postcode', 'Borough', 'Neighbourhood'])

Combine Neighbourhoods in same Postcode and drop resulting duplicate postcodes

In [4]:
for i,row in df.iterrows():
    if i==0: 
        continue
    if (df.at[i,'Postcode'] == df.at[i-1,'Postcode']):
        df.at[i,'Neighbourhood'] = df.at[i-1,'Neighbourhood'] + "," + df.at[i,'Neighbourhood']
        df.at[i-1,'Postcode'] = 'Delete'
                                     
df.drop(df[df.Postcode == 'Delete'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
print("Head of cleaned Dataframe: (top 20 rows)")
df.head(20)

Head of cleaned Dataframe: (top 20 rows)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [5]:
print("Check the size/shape of the dataframe:")
df.shape

Check the size/shape of the dataframe:


(103, 3)